In [ ]:
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

from dotenv import load_dotenv
import os

load_dotenv()

print(os.getenv("GROQ_API_KEY")) 

#llm
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0
)

# embeddings
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Vector DB
db = Chroma(
    collection_name="companyabt.docx",
    embedding_function=embedding,
    persist_directory=None  # Add this to avoid persistence issues
)


retriever = db.as_retriever()


prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question based on the context: {context}"),
    ("human", "{input}")
])


document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


response = retrieval_chain.invoke({"input": "What is the name of this company"})
print(response["answer"])